In [ ]:
# sftp_conn.makedirs('test/')


#THIS IS ULTIMATELY WRONG
# local_sftp_folder_name = '/bq_replications/savva_iota_file_transfer'

# IT SHOULD BE EXPORT_SFTP_FOLDER_NAME for the export
#What is the point of the target_sftp_folder_name here
#Write it out explicity for savva_export

# trying to change to directory SIS/ on savva SFTP  (target_sftp_folder_name)



# ------------------------FOR EXPORTS
#FOR savva

# target_sftp_folder_name = '/saava_iota_file_transfer'
# local_sftp_folder_name = '/bq_replications/savva_iota_file_transfer'
# export_sftp_folder_name = '/SIS'

# 'target_sftp_folder_name': '/clever_iota_file_transfer',
# 'local_sftp_folder_name': '/bq_replications/clever_iota_file_transfer',
# 'export_sftp_folder_name': '/home/boundless-calendar-0789', 

#source is the local_sftp_folder_name
#destination is the export_sftp_folder_name
# not sure why target_sftp_folder_name exists. Stale

# source is the local_sftp_folder_name
# destination is export_sftp_folder_name
# not sure why target_sftp_folder_name exists. Looks stale

#-----------------------FOR IMPORTS---

# For easyIEP

# 'target_sftp_folder_name': '/Reports',
# 'local_sftp_folder_name': '/easyIEP_combined', 
# 'export_SFTP_folder_name': None,

#source is the target_sftp_folder_name
#destination is the local_sftp_folder_name
#export_SFTP_folder_name is irrelevant.


# 'target_sftp_folder_name': '/idm-sensitive-exports',
# 'local_sftp_folder_name': '/powerschool_combined',
# 'project_id':'powerschool-420113'

#source is the target_sftp_folder_name
# destination is the local_sftp_folder_name
# #export_SFTP_folder_name is irrelevant.

#---------------------------------- NOTES

#Main issue is target_sftp_folder_name is being utilize in exports, and its useless
# I beleve export_sftp_folder_name should be used instea
# 
# See where target_sftp_folder_name is used in the exports.
# Seems like export_SFTP_folder_name could take over as target_sftp_folder_name
    
#Is the source

#For the import sftp_conn_source becomes the vendor



# Fixing Exports

In [ ]:
%load_ext autoreload
%autoreload 2
from modules.sftp_ops import *
from modules.sftp_utils import *
from modules.sftp_configs import * 

#Configure loggging
logging.basicConfig(filename='logs/SFTP_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New SFTP Operations Logging Instance')


sftp_type = 'clever_export'
local_sftp_type = 'iota_sftp'
import_or_export = 'export'
target_sftp_folder_name = '/home/boundless-calendar-0789'
local_sftp_folder_name = '/bq_replications/clever_iota_file_transfer'
naming_dict = clever_dictionary
db = 'roster_files'
export_local_bq_replications = True
project_id = 'powerschool-420113'

#specific conns retrieved
sftp_conn_obj = SFTPConnection.setup_sftp_connection(type_=sftp_type)
sftp_conn = sftp_conn_obj.get_connection()
logging.info(f'\n\n\nSFTP singular connection established successfully for {sftp_type}')

sftp_conn_local_obj = SFTPConnection.setup_sftp_connection(type_=local_sftp_type)
sftp_conn_local = sftp_conn_local_obj.get_connection()
logging.info(f'\n\n\nSFTP singular connection established successfully for {local_sftp_type}')

                



def SFTP_file_transfer(import_or_export, sftp_conn, sftp_conn_local, sftp_type, target_sftp_folder_name, local_sftp_folder_name, singular_file=None, naming_dict=None):


    # Set the source and destination connections based on import_or_export value
    if import_or_export == 'import':
        logging.info(f'Attempting to import SFTP files to local for {sftp_type}')
        sftp_conn_source = sftp_conn                      # Remote SFTP as the source
        sftp_conn_destination = sftp_conn_local           # Local SFTP as the destination
        fldr = target_sftp_folder_name
    elif import_or_export == 'export':
        logging.info(f'Attempting to export SFTP files to remote for {sftp_type}')
        sftp_conn_source = sftp_conn_local                # Local SFTP as the source
        sftp_conn_destination = sftp_conn                 # Remote SFTP as the destination
        fldr = local_sftp_folder_name
    else:
        raise ValueError(f'Invalid value for import_or_export: {import_or_export}')
    
    # Ensure destination directory exists
    ensure_sftp_directory_exists(sftp_conn_destination, local_sftp_folder_name)

    try:
        logging.info(f'Navigating to folder "{fldr}" for the {import_or_export} on {sftp_type}')

        sftp_conn_source.chdir(fldr)
        dir_contents = sftp_conn_source.listdir()
        logging.info(f'Dir contents of {fldr} for the {import_or_export} of {sftp_type}: {dir_contents}')
        
        if not dir_contents:
            logging.info(f'No files to download in folder for "{fldr}".')
            return

        # Handle singular file transfer
        if singular_file:
            if singular_file in dir_contents:
                dict_name = naming_dict.get(singular_file)
                source_file_path = os.path.join(target_sftp_folder_name, singular_file)
                destination_file_path = os.path.join(local_sftp_folder_name, dict_name)
                logging.info(f'Starting singular file transfer: "{singular_file}" -> "{destination_file_path}".')
                transfer_file(sftp_conn_source, sftp_conn_destination, source_file_path, destination_file_path)
                logging.info(f'Successfully transferred file "{singular_file}" to "{destination_file_path}".')
            else:
                logging.error(f'File "{singular_file}" not found in directory "{target_sftp_folder_name}".')
                raise AirflowException(f'File "{singular_file}" not found.')

        else:
            # Handle batch file transfer
            logging.info(f'No specification for file_to_download variable, assuming all files need to be transferred from {target_sftp_folder_name}')
            for file_name in dir_contents:
                source_file_path = os.path.join(target_sftp_folder_name, file_name)
                destination_file_path = os.path.join(local_sftp_folder_name, file_name)
                logging.info(f'Transferring file "{file_name}" to "{destination_file_path}".')
                transfer_file(sftp_conn_source, sftp_conn_destination, source_file_path, destination_file_path)
                logging.info(f'Successfully transferred file "{file_name}" to "{destination_file_path}".')

            logging.info(f'All files in folder "{target_sftp_folder_name}" transferred to destination SFTP.')

    except Exception as e:
        logging.info(f'Here is the error {e}')

    except IOError as e:
        logging.error(f'IOError occurred while accessing "{target_sftp_folder_name}" or transferring files: {e}')
        raise AirflowException(f'Error during SFTP operation: {e}')

    except Exception as e:
        logging.error(f'An error occurred during file replication {singular_file} attempt to {target_sftp_folder_name}: {e}')
        raise AirflowException
    
    logging.info(f'Finished SFTP file transfer from "{target_sftp_folder_name}" to "{local_sftp_folder_name}')

SFTP_file_transfer(import_or_export, 
                   sftp_conn, 
                   sftp_conn_local, 
                   sftp_type, 
                   target_sftp_folder_name, 
                   local_sftp_folder_name, 
                   singular_file=None, 
                   naming_dict=None)

# Updated

In [4]:
%load_ext autoreload
%autoreload 2
import sys
import os
import logging
from datetime import timedelta, datetime
# working_dir = os.path.join(os.environ['AIRFLOW_HOME'], 'git_directory/SFTP_Migrations')
# sys.path.append(working_dir)

from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
from modules.sftp_utils import *
from modules.sftp_configs import *
from modules.sftp_ops import *

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

#Configure loggging
logging.basicConfig(filename='logs/sftp_testing.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Logging Instance')


# --------------------------------------
for config in sftp_configs:

    task_id = f"sftp_file_exchange_{config['sftp_type']}"
    logging.info(f"Processing config for task {task_id}: {config}")


    SFTP_conn_file_exchange(**config)


#Defauted to downloading all files
#The files_to_download variable is being passed down as None not sure why

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
for filename in os.listdir('/home/local/imports'):
    print(filename)

PermissionError: [Errno 13] Permission denied: '/home/local/imports'